The goal of this notebook is to figure out a way to make the solving of differential equations faster 

In [1]:
import numpy as np 
import time
from scipy.integrate import solve_ivp
from MomentEquations import MomentsDiff_Eq_fn
from PredictionFunctions  import Get_Init_fn

Method 1 

In [2]:
nK = 14
times_arr = [0, 6, 12, 25, 45, 60, 90]
L  = np.array([10,15,20,25,50,250])*10**-3 #make it in nM
k = np.random.rand(nK)

In [3]:
# t_span2-tuple of floats
# Interval of integration (t0, tf). The solver starts with t=t0 and integrates until it reaches t=tf.

def solve_Moments_fn(K,IGF,t):
    """Inputs: K , L = IGF , tend
    Outputs: Z solution"""
    tspan = [0, t] 
    z0 = Get_Init_fn(K)
    sol_dyn = solve_ivp(MomentsDiff_Eq_fn, tspan, z0, method = 'BDF', args=(K,IGF))
    sol = sol_dyn.y[:,-1]
    return sol_dyn, sol
def FoxOn_preds_fn(k,L,t):
    """"Function spits out mean and second moment of nuclear foxO at time t
    input:  L = IGF concentration in nM 
            t = end time in seconds
    output: meanfoxO, variancefoxO"""
    _, Sol_t = solve_Moments_fn(k, L, t )
    foxOn_mean, foxOn_var = Sol_t[7], Sol_t[-1]
    # get the second moment from the variance 
    foxOn_s = foxOn_var + foxOn_mean**2
    return foxOn_mean, foxOn_s
# times_arr = np.array([0,6,15,21,51,126])*60
# L  = np.array([10,15,20,25,50,250])*10**-3 #make it in nM
def Moments_Preds_full_fn(k, times_arr, L):
    """Function that gets the predictions for 6 concentrations each at 6 time points """
    nL_cons = len(L)
    nCons = nL_cons*len(times_arr)*2 #number of constraints half of them means and half of them second moments 
    
    
    Moments_Preds_arr = np.zeros(nCons)
    i=0
    for igf in L: 
        ts = 0
        for t in times_arr:
#             print(t)
#             tint  = [ts, t]
#             print(tint)
#             print(tint)
                
            Moments_Preds_arr[i], Moments_Preds_arr[i + int(nCons/2)] = FoxOn_preds_fn(k, igf, t)
            i+=1
            ts = t

    return Moments_Preds_arr

In [4]:

t0 = time.time()
y1 = Moments_Preds_full_fn(k, times_arr,L)
print(time.time() - t0)
print(y1)

0.48627209663391113
[1.86820786 1.86799318 1.86799323 1.86799324 1.86799324 1.86799324
 1.86799324 1.86820786 1.86788777 1.8678877  1.86788775 1.86788776
 1.86788776 1.86788776 1.86820786 1.86778339 1.86778339 1.86778339
 1.86778339 1.86778339 1.86778339 1.86820786 1.86768016 1.8676802
 1.8676802  1.8676802  1.8676802  1.8676802  1.86820786 1.86718098
 1.86718083 1.86718095 1.867181   1.867181   1.86718101 1.86820786
 1.8639848  1.86398475 1.86398475 1.86398475 1.86398475 1.86398475
 3.93411421 3.93344387 3.93344403 3.93344407 3.93344407 3.93344407
 3.93344407 3.93411421 3.93311463 3.93311442 3.93311457 3.93311459
 3.93311459 3.93311459 3.93411421 3.93278859 3.93278859 3.93278858
 3.93278858 3.93278858 3.93278857 3.93411421 3.93246607 3.93246619
 3.93246618 3.93246618 3.93246618 3.93246618 3.93411421 3.93090581
 3.93090533 3.93090569 3.93090579 3.9309058  3.93090582 3.93411421
 3.92089055 3.92089041 3.9208904  3.9208904  3.9208904  3.92089041]


Method 2 

In [ ]:
# t_span2-tuple of floats
# Interval of integration (t0, tf). The solver starts with t=t0 and integrates until it reaches t=tf.

def solve_Moments_fn(K,IGF,tspan, z0):
    """Inputs: K , L = IGF , tend
    Outputs: Z solution"""
#     tspan = [0, t] 
#     z0 = Get_Init_fn(K)
    sol_dyn = solve_ivp(MomentsDiff_Eq_fn, tspan, z0, method = 'BDF', args=(K,IGF))
    sol = sol_dyn.y[:,-1]
    return sol_dyn, sol
def FoxOn_preds_fn(k,L,tspan, initial_conditions):
    """"Function spits out mean and second moment of nuclear foxO at time t
    input:  L = IGF concentration in nM 
            t = end time in seconds
    output: meanfoxO, variancefoxO"""
    _, Sol_t = solve_Moments_fn(k, L, tspan, z0 = initial_conditions )
    foxOn_mean, foxOn_var = Sol_t[7], Sol_t[-1]
    # get the second moment from the variance 
    foxOn_s = foxOn_var + foxOn_mean**2
    return foxOn_mean, foxOn_s, Sol_t

# times_arr = np.array([0,6,15,21,51,126])*60
# L  = np.array([10,15,20,25,50,250])*10**-3 #make it in nM
def Moments_Preds_full_fn(k, times_arr, L):
    """Function that gets the predictions for 6 concentrations each at 6 time points """
    nL_cons = len(L)
    nCons = nL_cons*len(times_arr)*2 #number of constraints half of them means and half of them second moments 
    z0 = Get_Init_fn(k)
    
    Moments_Preds_arr = np.zeros(nCons)
    i=0
    for igf in L: 
        ts = 0
        zinit = z0.copy()
        for t in times_arr:
            tspan  = [ts, t]

            Moments_Preds_arr[i], Moments_Preds_arr[i + int(nCons/2)], Sol_t = FoxOn_preds_fn(k, igf, tspan, initial_conditions = zinit)
            i+=1
            ts = t
            zinit = Sol_t
    return Moments_Preds_arr

In [ ]:

t0 = time.time()
y2 = Moments_Preds_full_fn(k, times_arr,L)
print(time.time() - t0)
print(y2)

In [ ]:
np.round(y2,3) == np.round(y1,3)